<b>324. Wiggle Sort II</b>

way 1: utilizing python builtin swap mechanism, if not allowed, need to think another way

In [1]:
def wiggleSort(nums):
    """same idea as above but concise"""
    nums.sort()
    k = len(nums[::2]) # if nums has odd number, k gets the extra 1, so left length >= right length
    # smaller number in even position
    # larger number in odd position
    # to avoid same number appear next to each other, reverse the element in small and large
    # like S, M,M,L => if not reverse, still S M M L, if reverse M L S M
    # the bellow take advantage of python's builting swapping machenism, otherwise cant guaranteer to change element 
    # in place while loop over it
    nums[::2], nums[1::2] = nums[:k][::-1], nums[k:][::-1]

In [2]:
nums = [1, 5, 1, 1, 6, 4]
wiggleSort(nums)
print(nums)

[1, 6, 1, 5, 1, 4]


way 2: same idea as above but not using builtin swapping, general to other language

In [3]:
def wiggleSort(nums):
    """
    Do not return anything, modify nums in-place instead.
    """
    """O(NlogN) and not o(1) space"""
    nums.sort()
    k = len(nums)//2 
    uneven = len(nums)%2
    small, large = nums[:k+uneven], nums[k+uneven:]
    # len(small) will always hold >= len(large)
    for i in range(len(large)):
        nums[2*i] = small[-i-1]
        nums[2*i+1] = large[-i-1]
    if uneven: # if extra number exist, it should belong to the left part
        nums[-1] = small[0]

In [4]:
nums = [1, 5, 1, 1, 6, 4]
wiggleSort(nums)
print(nums)

[1, 6, 1, 5, 1, 4]


<b>451.Sort Characters By Frequency </b>  
Counter() .most_common( ) => will return a list of tuple (item, count), no need sorted manually


In [5]:
from collections import Counter

def frequencySort(s):
    word_freq = Counter(s).most_common()
    return ''.join([char*count for char, count in word_freq])

In [6]:
frequencySort("tree")

'eetr'

<b>347 Top K Frequent Elements </b>. 

O(N) way
Collections.Counter( ). is O(N). Itertools.chain method is O(N) too  
First get {word: freq} dict, then create an empty bucket = [[] for _ in nums] the genius part is fill the bucket with bucket[-freq].append(word), the -freq will make sure the first non empty element in the bucket is the list of words with highest freq. chain method easily get away the empty list  

<b>ATTENTION</b>:  
DONT USE result = [ [ ] ] * len(nums)      
this step is wrong, all child[ ] ends up pointing to a same space  

In [15]:
from collections import Counter, defaultdict
from itertools import chain

def topKFrequent(nums, k):
    word_freq = Counter(nums) # {w: f}
    bucket = [[] for _ in nums]  # using len(nums), even though longer than needed, it is okay
    for word, freq in word_freq.items():
        bucket[-freq].append(word)  # genius step, -freq will ensure highest freq words are on the front of bucket
    return list(chain(*bucket))[:k] # chain() will merge all sub [] together and get rid of empty []

In [16]:
topKFrequent([1,1,1,2,2,3], 2)

[1, 2]

this way is smart and easy to explain => <b> remember this one </b>  
O(N)

In [1]:
def topKFrequent(nums):
    """Counter is O(N)"""
    count_dict = Counter(nums)
    freq_word_dict = defaultdict(list)
    for word, freq in count_dict.items():
        freq_word_dict[freq].append(word)

    """You can return the answer in any order. so if [1,1,1,2,2,3,3], k = 2
    my understanding is [1,2], [1,3] are both ok
    """
    l = len(nums)
    res = []
    for i in range(l, 0, -1): # max freq would be <=len(nums)
        if i in freq_word_dict and len(res)<k:
            res+=freq_word_dict[i]
    return res[:k]
            

O(NlogK) way of doing it. log K because after counter, it shrinks to k unique value

In [1]:
from collections import Counter
def topKFrequent(nums, k):
    # O(1) time 
    if k == len(nums):
        return nums
    # O(N)
    word_freq = Counter(nums)
#     word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)  # O(NLog(k))
    word_freq = word_freq.most_common()  # O(NLog(K))
    print(word_freq)
    result = [] 
    for i in range(k):
        result.append(word_freq[i][0])  # because question said k is always valid, so no index out of bound error
    return result
        

In [2]:
topKFrequent([1,1,1,2,2,3], 2)

[(1, 3), (2, 2), (3, 1)]


[1, 2]

<b>692 top K Frequent Words </b>  
Counter and sorted() function, sorted dictionary based on freq DESC and value asc

Also sort dictionary by values and then key and each based on different order type. 
sorted(a_dict.items(), key=lambda x: (-x[1], x[0]) )    -x[1] meaning order by value in desc
Or sorted(a_dict, key=lambda x: (-a_dict[x], x) ) => x is the key, when loop a_dict, key is the value iterated


In [37]:
 def topKFrequent(words, k):
    word_freq = Counter(words)
    word_freq = sorted(word_freq.items(), key=lambda x: (-x[1], x[0]))
    result = []
    for i in range(k): # question assumes k is always valid
        result.append(word_freq[i][0])
    return result

In [38]:
words = ["i", "love", "leetcode", "i", "love", "coding"]
k = 2
topKFrequent(words, k)

['i', 'love']

<b>1329.Sort the Matrix Diagonally </b>

Finding patterns of the question, here the element in the same diagonal has same value of index i - index J
So get same diagonal value out, sort, put them back. The temporary saving place’s add and pop method will affect the final big O

The reason to sorted in reverse order is that pop() method of list returns the last item.
So to make diagonal ascending, need to sort desc, then pop will reverse it back

In [48]:
from collections import defaultdict
def diagonalSort( mat):
    """
    append an item to a lsit in python is O(1), pop the last item from a list
    in python is O(1)
    2 nested for loop is O(NM)
    """
    m = len(mat)
    n = len(mat[0])
    diagonal_dict = defaultdict(list)   # a map to store all diagonal value
    for i in range(m):
        for j in range(n):
            diagonal_dict[i-j].append(mat[i][j])  # for item in the same diagonal, the i-j value is the same
    # sort each list inside of the diagonal_dict
    diagonal_dict = dict(map(lambda x: (x[0], sorted(x[1],reverse=True)), diagonal_dict.items()))
    for i in range(m):
        for j in range(n):
            mat[i][j] = diagonal_dict[i-j].pop()  # pop() returns the last item of the list, that is why REVERSE sort
    return mat

In [49]:
mat = [[3,3,1,1],[2,2,1,2],[1,1,1,2]]
diagonalSort(mat)

[[1, 1, 1, 1], [1, 2, 2, 2], [1, 2, 3, 3]]

<b> 791.Custom Sort String </b>

my first idea is using Counter() then construct the order based on S's order => O(N)
There are other ways that are more efficient

In [50]:
from collections import Counter
def customSortString(S, T):  
    """make T into dict"""
    t_dict = Counter(T)
    result = ""
    for s in S:
        if s in t_dict:  # O(1)
            result+= (s*t_dict.pop(s))
    for c in t_dict:
        result += (c*t_dict[c])
    return result

In [51]:
S = "cba"
T = "abcd"
customSortString(S,T)

'cbad'

<b>Ord()</b>: return a string representation of a char’s Unicode code, like old(‘a’) returns 97  
Python dictionary has .pop() function as well  
Using dictionary to help deciding the order  

In [ ]:
def customSortString(S, T):
    """using sort and ord() => but not too efficient, sorted() is O(NlogN)"""
    index_dict = {c:index for index, c in enumerate(S)}
    return ''.join(sorted(T, key=lambda x: index_dict.get(x, len(T)+ord(x))))

In [55]:
def customSortString_2(S, T):  
    """make S into dict, O(N)"""
    count = {}
    result = ""
    for s in S:
        count[s]=0
    for t in T:
        if t in S:
            count[t]+=1
        else:
            result+=t
    for s in S:
        result +=s*count[s]
    return result

In [56]:
S = "cba"
T = "abcd"
customSortString_2(S,T)

'dcba'

<b>49. Group Anagrams </b>

way 1: make a hashmap to store the words having same characters. the difference is how to construct the key.  
one way is to use the sorted word, which make the complexity to O(NKlogK) where N is the length of array, K is avg words length

In [57]:
def groupAnagrams(strs):
    hash_map = defaultdict(list)
    for w in strs:
        k=tuple(sorted(w))
        hash_map[k].append(w)
    return list(hash_map.values())

for constructing the key, i tried set(), list, they are not hashable, so cant be the key. 
tuple is hashable

In [58]:
strs = ["eat", "tea", "tan", "ate", "nat", "bat"]
groupAnagrams(strs)

[['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']]

way 2: if the count of each character is the same, then the words are anagram  
first construct a list of 26 0s, representing posiitons for 26 characters. loop through each word, add count up on corresponding positions of the list.  
Need to transform list into tuple to become the key of dictionary, because list is not hashbla.

using builtin function ord(), which returns the integer to represent the unicode point of the character.

The time complexity is O(NK) N is the length of array, K is the avg length of words

In [59]:
def groupAnagrams(strs):
    hash_map = defaultdict(list)
    for w in strs:
        k=[0]*26
        for c in w:
            k[ord(c)-ord('a')]+=1
        hash_map[tuple(k)].append(w)
    return list(hash_map.values())

In [60]:
strs = ["eat", "tea", "tan", "ate", "nat", "bat"]
groupAnagrams(strs)

[['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']]

<b>937. Reorder Data in Logs Files </b>  

This one is for come up with a helper function which can be used in the sorted() key param part.  
The helper function return a tuple, first value of the tuple is used for comparision first, second is used second, third is used third. if no other value provided, keep the original order

In [64]:
def reorderLogFiles(logs):
    def helper(log):
        # for each indivisual log, return a tuple of values used in sorted
        # first value in the tuple is used for comparison first, second then used second...
        id_, rest = log.split(" ", 1) # only split 1 time
        if rest[0].isalpha():
            # first char of the rest of logs
            return (0, rest, id_)
        else:
            return (1, )
    return sorted(logs, key=helper)    

In [65]:
logs = ["dig1 8 1 5 1","let1 art can","dig2 3 6","let2 own kit dig","let3 art zero"]
reorderLogFiles(logs)

['let1 art can',
 'let3 art zero',
 'let2 own kit dig',
 'dig1 8 1 5 1',
 'dig2 3 6']

<b>56.Merge intervals </b>. 

idea is to first sort the array using first item. then compare the adjacent elements. if they overlapped, merge them into one. otherwise, save the first element into the result and move forward for comparison  
sorting taks O(NlogN), comparison takes O(N), so total O(NlogN). 

using heapq is the same idea too

In [69]:
def merge(intervals):
    """Sorting takes O(n log(n)) and merging the intervals takes O(n). So, the resulting algorithm takes O(n log(n))."""
    # if not using heapq
    if not intervals:
        return []
    sorted_intervals = sorted(intervals, key=lambda x: x[0])
    result = []
    left = sorted_intervals[0]
    for right in sorted_intervals[1:]:
        if left[-1]>=right[0]:
            left[-1]=max(left[-1], right[-1])
        else:
            result.append(left)
            left = right
    result.append(left)
    return result

In [70]:
intervals = [[1,3],[3,6],[4,5],[15,18]]
merge(intervals)

[[1, 6], [15, 18]]

a more precise version:

In [ ]:
def merge(intervals):
    """Sorting takes O(n log(n)) and merging the intervals takes O(n). So, the resulting algorithm takes O(n log(n))."""
    """a precise version"""
    intervals.sort(key = lambda x: x[0])
    result = []

    for item in intervals:
        if not result or result[-1][-1]<item[0]:
            # if result is empty (just start)
            # or NO overlapping
            result.append(item)
        else:
            result[-1][-1] = max(result[-1][-1], item[-1])
    return result

In [67]:
intervals = [[1,3],[3,6],[4,5],[15,18]]
merge(intervals)

[[1, 6], [15, 18]]

<b>57 insert interval </b>  
similar to the problem 56 merge intervals

way 1: my way, update the newInterval when merge with existing one, so it can be used to compare with next item in the intervals.


In [1]:
def insert(intervals, newInterval):
    result = []
    for ind, item in enumerate(intervals):
        if item[0] > newInterval[1]:
            result.append(newInterval)
            result.extend(intervals[ind:])
            return result  # => here no need to loop other items in the intervals since the newInterval is merged already
        elif item[1]<newInterval[0]:
            result.append(item)
        else:
            newitem = [min(item[0], newInterval[0]), max(item[1], newInterval[1])]
            newInterval = newitem
    result.append(newInterval) # can take care empty intervals case, and the case when last item left is the newInterval
    return result

way 2: from da shen. loop over each item, update the start and end, which is upating the new interval too

In [2]:
def insert(intervals, newInterval):
      
    start, end = newInterval[0], newInterval[1]
    left, right = [], []
    for item in intervals:
        if item[1] < start:
            left.append(item)
        elif item[0]>end:
            right.append(item)
        else:
            start = min(start, item[0])
            end = max(end, item[1])
    return left+[[start, end]]+right

inspire by the way 2, i update the way 1 method, with make it faster!

In [5]:
def insert(intervals, newInterval):
    # using start, and end instead of updating the newInterval var every time after updating start and end  
    start, end = newInterval[0], newInterval[1]
    result = []
    for ind, item in enumerate(intervals):
        if item[0] > end:
            result.append([start, end])
            result.extend(intervals[ind:])
            return result  # => here no need to loop other items in the intervals since the newInterval is merged already
        elif item[1]<start:
            result.append(item)
        else:
            start, end = min(item[0], start), max(item[1], end)
    result.append([start, end]) # can take care empty intervals case, and the case when last item left is the newInterval
    return result

In [3]:
intervals = [[1,3],[6,9]]
newInterval = [2,5]
insert(intervals, newInterval)

[[1, 5], [6, 9]]

<b>252.Meeting room </b>  

logic is the same as above


In [73]:
def canAttendMeetings(intervals):
    if not intervals:
        return True
    intervals.sort(key=lambda x: x[0])
    last_meeting = intervals[0]
    for next_meeting in intervals[1:]:
        if last_meeting[-1]>next_meeting[0]:
            return False
        else:
            last_meeting = next_meeting
    return True

In [74]:
intervals = [[0,30],[5,10],[15,20]]
canAttendMeetings(intervals)

False

In [75]:
def canAttendMeetings(intervals):
    if not intervals:
        return True
    intervals.sort(key=lambda x: x[0])
    n = len(intervals)
    for i in range(n-1):
        if intervals[i][-1]>intervals[i+1][0]:
            return False
    return True

In [76]:
intervals = [[0,30],[5,10],[15,20]]
canAttendMeetings(intervals)

False

<b>435.Non-overlapping Intervals</b>  
sort by end is the key, keep the intervals ends first, so latter ones can have more room, it will require less amount of items to be removed compare with sort by start.

In [5]:
def eraseOverlapIntervals(intervals) :
    """find max number of items that are not overlapping"""
    if not intervals:
        return 0
     # sort by last number, so can find MIN number of intervals need to remove
        # keep the one ends early will give latter one more room to not overlap
    intervals.sort(key = lambda x: x[1]) 
    start = intervals[0]
    count = 1 # count is the non-overlapping count
    for end in intervals[1:]:
        if start[-1]<=end[0]:
            count += 1
            start = end
    return len(intervals)-count

In [6]:
def eraseOverlapIntervals(intervals):
    """find max number of items that are not overlapping"""
    if not intervals:
        return 0
    # sort by last number, so can find MIN number of intervals need to remove

    # keep the one ends early will give latter one more room to not overlap
    intervals.sort(key = lambda x: x[1]) 
    end = float('-inf')
    result = 0
    for i in intervals:
        if i[0]>=end:
            # no overlapping, keep i
            end = i[1]
        else:
            # overlapping occurs
            result+=1
    return result

<b> 731 My calendar II </b>

one array for each event, another array for overlapped interval.  
the statement to check overlapped is smart.  
Time is O(N^2): O(N) for each book operation and there are N booking operations. So N * N


In [8]:
class MyCalendarTwo:

    def __init__(self):
        self.overlaps =[]  # [(start, end)] save overlapped intervals
        self.calendar = [] # [(start, end)] save booked events

    def book(self, start: int, end: int) -> bool:
        # check if double booked or not
        for i, j in self.overlaps:
            if start < j and end >i:
                # the above statement takes care of three scenarios:
                # i start j end; i start, end, j; start, i, end, j
                # only eliminate the case: start, end, i, j
                # my original though is start<j and start>i, but it wont take care of the case: start, i, end, j 
                return False
        for i, j in self.calendar:
            if start <j and end >i:
                # update the overlap:
                self.overlaps.append((max(start,i), min(end,j)))
        # no matter what, update the calendar list
        self.calendar.append((start, end))
        return True


# Your MyCalendarTwo object will be instantiated and called as such:
# obj = MyCalendarTwo()
# param_1 = obj.book(start,end)

In [11]:
MyCalendar = MyCalendarTwo()
print(MyCalendar.book(10, 20)) # returns true
print(MyCalendar.book(50, 60)) # returns true
print(MyCalendar.book(10, 40)) # returns true
print(MyCalendar.book(5, 15)) # returns false
print(MyCalendar.book(5, 10)) # returns true
print(MyCalendar.book(25, 55)) # returns true

True
True
True
False
True
True


using way in calendar III but very slow because of the sort

In [4]:
from collections import defaultdict
class MyCalendarTwo:

    def __init__(self):
        self.calendar = defaultdict(int) # {time: int} +1 for start, -1 for end

    def book(self, start: int, end: int) -> bool:
        
        self.calendar[start]+=1
        self.calendar[end]-=1
        ongoing = 0
        max_ongoing = 0
        for t in sorted(self.calendar.keys()):
            ongoing += self.calendar[t]
            max_ongoing = max(max_ongoing, ongoing)
            if max_ongoing >=3:
                # revert adding the interval to calendar
                self.calendar[start]-=1
                self.calendar[end]+=1
                return False
        return True    


<b> 732. My Caldendar III </b>

A totally different way than my calendar II.  
Count the ongoing meetings, need to sort the meeting by start time, or making sure when a new time is added, the event’s starting time is sorted. In this way the counting can make sense 

for example: 
["MyCalendarThree","book","book","book","book","book","book"]  
[[],[10,20],[50,60],[10,40],[5,15],[5,10],[25,55]]  

[10, 20] => k: 1, 0 > 1  
[10,20],[50,60] => k:1, 0, 1, 0 => 1  
[10,20],[50,60],[10,40] => [10,10, 20, 40,50,60 ] => k: 1,2,1,0,1,0 => 2  
[10,20],[50,60],[10,40],[5,15] => [5,10,10,15,20,40,50,60] => k:1,2,3,2,1,0,1,0 => 3


In [2]:
from collections import defaultdict
class MyCalendarThree:

    def __init__(self):
        self.time_line = defaultdict(int) # {'time': 1 (start) or -1 (end)}

    def book(self, start: int, end: int) -> int:
        self.time_line[start] += 1
        self.time_line[end]-=1
        ongoing = 0
        k = 0
        for t in sorted(self.time_line.keys()):
            ongoing += (self.time_line[t])
            k = max(k, ongoing)
            
        return k
        

using bisect.insort to ensure the new start and end are added in a sorted way

In [3]:
from bisect import insort
class MyCalendarThree:

    def __init__(self):
        self.time_line = [] # (time, 1 or -1)

    def book(self, start: int, end: int) -> int:
        # ensure start and end are added to still keep time_line sorted
        insort(self.time_line, (start, 1))
        insort(self.time_line, (end, -1))
        ongoing = 0
        k = 0
        for t, v in self.time_line:
            ongoing += v
            k = max(k, ongoing)
            
        return k
        

<b> 253.Meeting Rooms II </b>

In [7]:
def minMeetingRooms(intervals):
    starts = sorted(i[0] for i in intervals)
    ends = sorted(i[1] for i in intervals)
    available = room_needed = 0
    s = 0 # start meeting pointer
    e = 0  # end meeting pointer
    while s < len(intervals):
        # need a room
        if starts[s] < ends[e]:
            if available > 0:
                available -= 1
            else:
                room_needed += 1
            s+=1
        else:
            # a previous meeting is ended before the next one starts:
            # it will keep checking, and it works becauses when s == e, it will need a room
            available += 1
            e+=1
    return room_needed

In [8]:
intervals=[[0,2],[1,3],[2,4],[3,5],[4,6]]
minMeetingRooms(intervals)

2

<b> Merge Sort</b>

<b> 315 Count of Smaller Numbers After self </b>

In [6]:
def countSmaller(nums):
    """bisection, mergesort, binary tree"""

    def mergesort(nums, indexes, start, end):
        # start, end index
        # indexes: a list of index
        if end <= start:
            return
        mid = (start+end)//2 # split the array into half
        mergesort(nums, indexes, start, mid)
        mergesort(nums, indexes, mid+1, end)

        merge(nums, indexes, start, end)

    def merge(nums, indexes, start, end):
        mid = (start+end)//2
        left = start
        right = mid+1
        right_count = 0
        # the order of indexes after merge sort, only for index from start to end
        new_indexes = [0]*(end-start+1)

        sort_index = 0 # keep track of position for the new index array
        while left<=mid and right<=end:
            # merge sorting, if element in the right < element in the left
            # find an element on the right side smaller than the element in the left side
            if nums[indexes[right]] < nums[indexes[left]]:
                # merge sort result, keep the smaller item's index in a new index list
                new_indexes[sort_index] = indexes[right]
                # ??
                right_count+=1
                # move to next item in the right part
                right += 1
            else:
                # left element is smaller, updates its count
                new_indexes[sort_index] = indexes[left]
                # this is the important step
                # in the part of [start, end], for value in position index[left], so far found right_count this amount of items smaller than it
                # += because in the other part of array, like mid to end, there can be other items smaller than value in position index[left]
                count[indexes[left]] += right_count
                left+=1
            # merge sort next position
            sort_index+=1
        while left <= mid:
            new_indexes[sort_index] = indexes[left]
            count[indexes[left]] += right_count
            left += 1
            sort_index+=1
        while right<=end:
            new_indexes[sort_index] = indexes[right]
            right+=1
            sort_index+=1
        for i in range(start, end+1):
            indexes[i]=new_indexes[i-start]


    count = [0]*len(nums)
    indexes = list(range(len(nums)))
    mergesort(nums,indexes, 0, len(nums)-1)

    return count

In [7]:
nums = [5,2,6,1]
countSmaller(nums)

[2, 1, 1, 0]

same idea but in a more concise way

In [ ]:
def countSmaller(nums):
        """mergesort"""
    def sort_helper(indexes):
        half = len(indexes) // 2
        if half:
            left, right = sort_helper(indexes[:half]), sort_helper(indexes[half:])
            for i in range(len(indexes))[::-1]:
                if not right or left and nums[left[-1]] > nums[right[-1]]:
                    smaller[left[-1]] += len(right)
                    indexes[i] = left.pop()
                else:
                    indexes[i] = right.pop()
        return indexes
    smaller = [0] * len(nums)
    sort_helper(list(range(len(nums))))
    return smaller


<b> 179 largest number </b>

this problem tests different types of sort. the compare function that adds two numbers in string format then compare is the key to address the order 

bubble sort:  
Repeatedly swapping the adjacent elements if they are in wrong order  
Worse case: O(N^2)  
Average case: O(N^2)  
Best case: O(N)  
 

In [ ]:
def helper_compare(n1, n2):
    return str(n1)+str(n2) > str(n2)+str(n1)

In [1]:
def largestNumber(nums):
    """bubble sort O(N^2), move smallest item in the last position"""
    for i in range(len(nums)+1, 1, -1): # i in [l+1, 2]
        for j in range(1, i-1): # [1, l-1] => [1, 1]=> []
            if not helper_compare(nums[j-1], nums[j]):
                nums[j-1], nums[j] = nums[j], nums[j-1]
    res = "".join(map(str, nums)).lstrip("0")
    return res if res else "0" 

In [2]:
nums = [3,30,34,5,9]
largestNumber(nums)

'9534330'

selecton sort:  
It is a in-place comparison sorting also. finding the smallest element in the unsorted sublist, exchanging (swapping) it with the leftmost unsorted element (putting it in sorted order), and moving the sublist boundaries one element to the right. So it maintains two subarray: one is sorted, another is still unsorted

Here it find the smallest item in the left unsorted part, and replace with the right part.
rightest position is the first to be replaced.

worst, best, avg:  O(N^2)

In [1]:
def largestNumber(nums):
    """selection sort, maintain two list, one is sorted, the other is unsorted. keep finding the smallest element in the unsorted sublist, swap to put it in the right position to extend the sorted list"""
    def helper_compare(n1, n2):
        return str(n1) + str(n2) > str(n2)+str(n1)
    for i in range(len(nums), 1, -1): # [l, 2]
        tmp = 0 # save index for smallest item
        for j in range(i): # [0, l-1], [0, 1]
            if not helper_compare(nums[j], nums[tmp]):
                tmp = j
        nums[tmp], nums[i-1] = nums[i-1], nums[tmp]
    res = "".join(map(str, nums)).lstrip("0")
    return res if res else "0" 



merge sort:  
split array in half until only one item left, then compare and merge.  
O(NlogN)

In [2]:
def largestNumber(nums):
        """merge sort"""
        def compare(n1, n2):
            """n1, n2 is the item"""
            return str(n1)+str(n2) > str(n2)+str(n1)
        def mergeSort(nums, l, r):
            """split array until one element, then merge from bottom
            l,r are the valid index
            """
            
            # base case:
            if l > r:
                return
            if l == r:
                return [nums[l]]  # why return a list of element=> left or right need to be a list, because merge() takes in list
            mid = l+(r-l)//2
            left = mergeSort(nums, l, mid)
            right = mergeSort(nums, mid+1, r)
            return merge(left, right)
        
        def merge(l_array, r_array):
            """merge two array in descending order"""
            res, i, j = [], 0, 0
            while i<len(l_array) and j<len(r_array):
                if compare(l_array[i], r_array[j]):
                    # l_array[i] > r_array[j]
                    res.append(l_array[i])
                    i+=1
                else:
                    res.append(r_array[j])
                    j+=1
            # it can happen that l_array or r_array has not finished yet
            res.extend(l_array[i:] or r_array[j:])
            return res        
                
        sorted_num = mergeSort(nums, 0, len(nums)-1)    
        res = "".join(map(str, sorted_num)).lstrip("0")
        return res if res else "0"

            

quick sort:  
having a pivot value, all values in the left of the pivot position should be larger than it, all values in the right side of the pivot position should be smaller than it.   
ave: O(NlogN). 
worst: O(N^2)

In [4]:
def largestNumber(nums):
    """quick sort, having a partition"""
    def compare(n1, n2):
        """n1, n2 are the real item in the array"""
        return str(n1)+str(n2) > str(n2)+str(n1)
    def partition(nums, l, r):
        """l, r are valid index"""
        low = l
        pivot = nums[r]
        for i in range(l, r):
            if compare(nums[i], pivot):
                nums[low], nums[i] = nums[i], nums[low]
                low += 1
        nums[low], nums[r] = nums[r], nums[low]
        return low
    def quickSort(nums, l, r):
        if l >= r:
            # in-place sort has finished
            return
        pos = partition(nums, l, r)
        quickSort(nums, l, pos-1)
        quickSort(nums, pos+1, r)

    quickSort(nums, 0, len(nums)-1)
    res = "".join(map(str, nums)).lstrip("0")
    return res if res else "0"
        

<b> 767 reorganize string </b>

the key is to use two sorts, first make sure the char in lexical order, secondly sort by its count! so "aab" => "baa".  
Then put the first half in odd index place, last half in even index place. Using python’s inplace switch.


In [5]:
def reorganizeString(S):
    l = len(S)
    if not l:
        return ""
    # first sort by char lexical order, then sort by count of chars
    # can solve the case like "aab", if not sort by count, wont get the right answer
    # the two sorts are the key
    new_s = sorted(sorted(S), key=S.count)
    mid = l//2
    new_s[1::2], new_s[::2] = new_s[:mid], new_s[mid:]
    # new_s[-1:]!=new_s[-2:-1] returns 0 or 1
    return ''.join(new_s) * (new_s[-1:]!=new_s[-2:-1]) 

In [6]:
reorganizeString("aab")


'aba'

another way only takes O(N) but uses a little bit math
if the max count of a char is larger than (len(S)+1)//2, then it is an impossible result

len 3, max count 2  
len 4, max count 2  
len 5, max count 3  
len 6, max count 3  

In [1]:
from collections import Counter
def reorganizeString(S):
    """O(N) way, find the char with most count, put it in even position, then put other chars"""
    count_dict = Counter(S)
    max_count = 0
    max_c = None
    for c, count in count_dict.items():
        if count > max_count:
            max_count = count
            max_c = c
    if max_count > ((len(S)+1)//2):
        # impossible case
        return ""

    # put the char with most count in EVEN position first
    ind = 0
    res = [0]*len(S)
    while count_dict[max_c]>0:
        res[ind] = max_c
        ind+=2
        count_dict[max_c]-=1
    for char in count_dict:
        while count_dict[char]>0:
            if ind >= len(S):
                ind = 1 # back to the starting odd position, this will only appear once
            res[ind] = char
            ind+=2
            count_dict[char] -= 1   
    return "".join(res)


In [2]:
reorganizeString("aab")

'aba'